In [1]:
include("../src/industrial_stats.jl")
using .IndustrialStats

# Overview
The goal is to build a runner that distributes a bunch of batch jobs across multiple processors on the host to speed up the computation time.

The user must supply a function that handles a batch and another function that, given the current batch number and batch size, produces an input for the function that handles each job.

Then based on the parameters provided by the user the runner will spin up a certain number of processes and distribute the jobs over these, invoking the batch function each time a job finishes on any of the processors.

# Settings

In [1]:
NUM_PROCS = 4

4

# Imports

In [2]:
# Set the number of processes to use for parallel computing
using Distributed
addprocs(NUM_PROCS)

4-element Vector{Int64}:
 2
 3
 4
 5

In [7]:
using HDF5
using Dates
using Logging
using IterTools

# These imports are distributed to all processes
@everywhere include("../src/industrial_stats.jl")
@everywhere using .IndustrialStats
@everywhere using LinearAlgebra
@everywhere using Optim
@everywhere using ForwardDiff
@everywhere using Optimization, OptimizationNLopt

      From worker 4:	WARNING: replacing module IndustrialStats.
      From worker 5:	WARNING: replacing module IndustrialStats.
      From worker 3:	WARNING: replacing module IndustrialStats.
      From worker 2:	WARNING: replacing module IndustrialStats.


In [8]:
function batch_matrix(matrix::Array{Float32, 3}, idx, batch_size)::Array{Float32, 3}
    size_check = size(matrix, 1)
    return matrix[idx:min(idx + batch_size - 1, size_check), :, :]
end

function batch_matrix_generator(matrix::Array{Float32, 3})::Function
    (idx, batch_size) -> batch_matrix(matrix, idx, batch_size)
end

batch_matrix (generic function with 1 method)